In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
import dcube as dc
import statsmodels.api as sm
import datetime
from dateutil.relativedelta import relativedelta
import os

%run Quant_useful_packages_ccb.ipynb
from Quant_useful_packages import get_near_trade_day


pro = dc.pro_api('')


In [6]:
cache_data_dir = 'C:/Users/nibh/Desktop/FZ/CCB'


def get_last_report_period(date,backyear=1):
    date = pd.to_datetime(date).date()
    year = pd.to_datetime(date).year
    month = pd.to_datetime(date).month
    month_fristday = datetime.date(date.year,date.month,1)
    report_period_list = []
    for i in range(12*backyear):   # 按月回溯
        premonth_lastday = month_fristday-relativedelta(months=i)-relativedelta(days=1)   # 回溯月的上个月的最后一天
        premonth_year = premonth_lastday.year
        premonth_month = premonth_lastday.month
        preyear = year - backyear + 1
        if premonth_year < preyear and premonth_month < 9:
            break
        if premonth_lastday.month % 3== 0:
            report_period_list.append(premonth_lastday.strftime('%Y%m%d'))
    return report_period_list[::-1]

    
def refresh_fin_byQ():
    def get_fin_byQ_dict(start_date,end_date):
        start_date = pd.to_datetime(start_date).date()
        end_date = pd.to_datetime(end_date).date()
        backyear = end_date.year - start_date.year + 4
        report_period_list = get_last_report_period(end_date,backyear)
        print(report_period_list)
        df = []
        map_dic = {}
        map_dic['s_qfa_roe_deducted'] = 's_qfa_roe_deducted'
        map_dic['s_qfa_eps'] = 's_qfa_eps'
        factor_list = list(map_dic.keys())
        fields = 'code,re_period,ann_date'
        for f in factor_list:
            fields = fields+',{}'.format(f)
    #     print(fields)
        for report_period in report_period_list:
            df_tmp = pro.query('fin_id', fields=fields,re_period=report_period)
            df.append(df_tmp)
        df = pd.concat(df,axis=0)
        # print(df)
        df['re_period1'] = df['re_period']

        index = ['code','re_period']
        # ann_date_df = df.groupby(index).head(1).set_index(index).unstack()['ann_date']
        # re_period_df = df.groupby(index).head(1).set_index(index).unstack()['re_period1']

        q_dict = {}
        rolling_factor_list = ['s_qfa_roe_deducted','s_qfa_eps']
        rolling_factor_list = list(set(rolling_factor_list)&(set(factor_list)))  # 取交集
        for f in factor_list:
            factor_df = df.groupby(index).head(1).set_index(index).unstack()[f]  # 每个可转债对应的第一个该指标
            q_dict[f] = factor_df
            if f in rolling_factor_list:
                q_dict['{}_pre1q'.format(f)] = factor_df.shift(1,axis=1)
                # q_dict['{}_pre2q'.format(f)] = factor_df.shift(2,axis=1)
                # q_dict['{}_pre3q'.format(f)] = factor_df.shift(3,axis=1)
                q_dict['{}_pre4q'.format(f)] = factor_df.shift(4,axis=1)
                # q_dict['{}_pre5q'.format(f)] = factor_df.shift(5,axis=1)

                factor_df_8qmean = factor_df.rolling(8,axis=1).mean()
                factor_df_8qstd = factor_df.rolling(8,axis=1).std()
                factor_df_8qzscore = ((factor_df - factor_df_8qmean)/factor_df_8qstd.replace(0,np.nan)).fillna(0)
                q_dict['{}_8qzscore'.format(f)] = factor_df_8qzscore


                factor_df_diff4q = factor_df - factor_df.shift(4,axis=1)
                factor_df_diff4q_8qmean = factor_df_diff4q.rolling(8,axis=1).mean()
                factor_df_diff4q_8qstd = factor_df_diff4q.rolling(8,axis=1).std()
                factor_df_diff4q_8qzscore = ((factor_df_diff4q - factor_df_diff4q_8qmean)/factor_df_diff4q_8qstd.replace(0,np.nan)).fillna(0)
                q_dict['{}_diff4q'.format(f)] = factor_df_diff4q
                # q_dict['{}_diff4q_8qmean'.format(f)] = factor_df_diff4q_8qmean
                # q_dict['{}_diff4q_8qstd'.format(f)] = factor_df_diff4q_8qstd
                q_dict['{}_diff4q_8qzscore'.format(f)] = factor_df_diff4q_8qzscore

                factor_df_diff1q = factor_df - factor_df.shift(1,axis=1)
                factor_df_diff1q_8qmean = factor_df_diff1q.rolling(8,axis=1).mean()
                factor_df_diff1q_8qstd = factor_df_diff1q.rolling(8,axis=1).std()
                factor_df_diff1q_8qzscore = ((factor_df_diff1q - factor_df_diff1q_8qmean)/factor_df_diff1q_8qstd.replace(0,np.nan)).fillna(0)
                q_dict['{}_diff1q'.format(f)] = factor_df_diff1q
                q_dict['{}_diff1q_8qzscore'.format(f)] = factor_df_diff1q_8qzscore
        return q_dict



    init_start_date = '20041201'
    pre_date = datetime.date.today() - datetime.timedelta(days=1)
    pre_date = get_near_trade_day(pre_date).strftime('%Y%m%d')
    all_days = get_all_trade_days()
    all_days = [date.strftime('%Y%m%d') for date in all_days if date.strftime('%Y%m%d')>init_start_date]
    df_tmp = pd.DataFrame()
    start_date = init_start_date
    end_date = pre_date
    q_dict = get_fin_byQ_dict(start_date,end_date)
    factor_list = q_dict.keys()
    df_tmp = None
    for factor in factor_list:
        suffix = 'byQ'
        file_name = '{}/{}_{}.csv'.format(cache_data_dir,factor,suffix)
        if_exist = os.path.exists(file_name)
        if if_exist:
            df = pd.read_csv(file_name,index_col=0)
            df.index = [str(x) for x in df.index]
            df.index = pd.to_datetime(df.index)
            last_date = df.index.tolist()[-1].strftime('%Y%m%d')
#             print("pre_date={},last_date={}".format(pre_date,last_date))
            if True:
                date_list_tmp = [date for date in all_days if last_date<date<=pre_date]
                df_tmp = q_dict[factor].T
                df_tmp.index = pd.to_datetime(df_tmp.index)
                remain_index = [d for d in df.index if d not in df_tmp.index]
                df = df.reindex(remain_index).append(df_tmp).sort_index()
                df.to_csv(file_name)
                print('数据{}_{}更新完毕'.format(factor,suffix))
            else:
                print('数据{}_{}已是最新'.format(factor,suffix))
        else:
            date_list_tmp = [date for date in all_days if init_start_date<=date<=pre_date]
            df_tmp = q_dict[factor].T
            df_tmp.index = pd.to_datetime(df_tmp.index)
            df_tmp.to_csv(file_name)
            print('数据{}_{}更新完毕'.format(factor,suffix))
    return df_tmp


def refresh_fin_eod_from_Q():
    init_start_date = '20101201'
    pre_date = datetime.date.today() - datetime.timedelta(days=1)
    pre_date = get_near_trade_day(pre_date).strftime('%Y%m%d')
    all_days = get_all_trade_days()
    all_days = [date.strftime('%Y%m%d') for date in all_days if date.strftime('%Y%m%d')>init_start_date]
    df_tmp = pd.DataFrame()
    start_date = init_start_date
    end_date = pre_date
    ori_factor_list = ['s_qfa_roe_deducted','s_qfa_eps']
    suffix_list = ['_pre1q','_pre4q',\
                    '_8qzscore',\
                    '_diff4q','_diff4q_8qzscore',\
                    '_diff1q','_diff1q_8qzscore',\
                    ][:]
    factor_list = ['{}{}'.format(f,suffix) for f in ori_factor_list for suffix in suffix_list]
    print('refresh_fin_eod_from_Q...:start_date:{},end_date:{}'.format(start_date,end_date))
    re_period_df = get_single_factor_values('re_period',start_date,end_date)
    re_period_df = re_period_df.fillna(0).astype(int).astype(str)[pd.notnull(re_period_df)]
    re_period_date_list = re_period_df.index.tolist()
    @lru_cache(1)
    def read_eod_file(file_name):
        df = pd.read_csv(file_name,index_col=0)
        df.index = [str(x) for x in df.index]
        df.index = pd.to_datetime(df.index)
        return df
    for factor in factor_list:
        print('refresh_incom_eod_from_Q...:{}'.format(factor))
        file_name_byQ = '{}_byQ'.format(factor)
        file_name = '{}/{}.csv'.format(cache_data_dir,factor)
        if_exist = os.path.exists(file_name)
        if if_exist:
            df = read_eod_file(file_name)
            last_date = pd.to_datetime(df.index.tolist()[-1].strftime('%Y%m%d'))
            date_list = [date for date in re_period_date_list if date>=last_date]
        else:
            date_list = re_period_date_list

        df_byQ = read_single_factor_file(file_name_byQ)
        df_byQ = df_byQ.dropna(axis=0,how='all')
        df_byQ.index = [date.strftime('%Y%m%d') for date in df_byQ.index]
        df_byQ = df_byQ.to_dict()
        df_eod = {}

        for date in date_list:
            if date_list.index(date)%250==0:
                print(date)
            factor_oneday = {}
            for s in re_period_df.columns:
                report_q = re_period_df.loc[date,s]
                factor_oneday[s] = df_byQ.get(s,{}).get(report_q,np.nan)
            df_eod[date] = factor_oneday
        df_eod = pd.DataFrame(df_eod).T
        df_eod.index = pd.to_datetime(df_eod.index)
        if if_exist:
            df = read_eod_file(file_name)
            last_date = df.index.tolist()[-1].strftime('%Y%m%d')
            if True:
                date_list_tmp = [date for date in all_days if last_date<date<=pre_date]
                df_tmp = df_eod.copy()
                df_tmp.index = pd.to_datetime(df_tmp.index)
                remain_index = [d for d in df.index if d not in df_tmp.index]
                df = df.reindex(remain_index).append(df_tmp).sort_index()
                df.to_csv(file_name)
                print('数据{}更新完毕'.format(factor))
            else:
                print('数据{}已是最新'.format(factor))
        else:
            date_list_tmp = [date for date in all_days if init_start_date<=date<=pre_date]
            df_tmp = df_eod.copy()
            df_tmp.index = pd.to_datetime(df_tmp.index)
            df_tmp.to_csv(file_name)
            print('数据{}更新完毕'.format(factor))
    return df_tmp




def refresh_re_period_eod():
    def get_re_period_dict(start_date,end_date):
        start_date = pd.to_datetime(start_date).date()
        end_date = pd.to_datetime(end_date).date()
        backyear = end_date.year - start_date.year + 1
        report_period_list = get_last_report_period(end_date,backyear)[:]
        print(report_period_list)
        df = []
        map_dic = {}
        map_dic['ann_date'] = 'ann_date'
        map_dic['re_period'] = 're_period'
        factor_list = list(map_dic.keys())
        fields = 'code,re_period,ann_date'
        for report_period in report_period_list:
            df_tmp = pro.query('fin_id', fields=fields,re_period=report_period)
            df.append(df_tmp)
        df = pd.concat(df,axis=0)
        df['re_period1'] = df['re_period']
        index = ['code','re_period']
        ann_date_df = df.groupby(index).head(1).set_index(index).unstack()['ann_date']
        re_period_df = df.groupby(index).head(1).set_index(index).unstack()['re_period1']
        eod_dict = {}
        eod_dict['ann_date'] = ann_date_df
        eod_dict['re_period'] = re_period_df

        year_list = sorted(list(set([c[:4] for c in re_period_df.columns])))

        date_e = end_date
        date_s = datetime.date(int(year_list[0])+1,1,1)
        all_days = get_all_trade_days()
        date_list = [date for date in all_days if date_s<=date<=date_e]
        eod_df_dict = {}

        for f,factor_df in eod_dict.items():
            eod_df_dict[f] = {}
        for date in date_list:
            if date_list.index(date)%250 == 0:
                print('get_re_period_dict......{}'.format(date))
            ann_date_flag = (ann_date_df.astype(float)<=float(date.strftime('%Y%m%d')))\
                        & (~(ann_date_df.astype(float).shift(-1,axis=1)<=float(date.strftime('%Y%m%d'))))
            for f,factor_df in eod_dict.items():
                eod_df_dict[f][date] = factor_df[ann_date_flag].fillna(method='ffill',axis=1).iloc[:,-1]
        for f,factor_df in eod_dict.items():
            eod_df = pd.DataFrame(eod_df_dict[f]).T.loc[start_date:end_date]
            eod_df.index = pd.to_datetime(eod_df.index)
            eod_df_dict[f] = eod_df
        return eod_df_dict

    init_start_date = '20101201'
    pre_date = datetime.date.today() - datetime.timedelta(days=1)
    pre_date = get_near_trade_day(pre_date).strftime('%Y%m%d')
    all_days = get_all_trade_days()
    all_days = [date.strftime('%Y%m%d') for date in all_days if date.strftime('%Y%m%d')>init_start_date]
    df_tmp = pd.DataFrame()
    start_date = init_start_date
    end_date = pre_date
    # start_date = '20041201'
    # end_date = '20081201'
    # start_date = '20151201'
    eod_df_dict = get_re_period_dict(start_date,end_date)
    factor_list = eod_df_dict.keys()
    for factor in factor_list:
        file_name = '{}/{}.csv'.format(cache_data_dir,factor)
        if_exist = os.path.exists(file_name)
        if if_exist:
            df = pd.read_csv(file_name,index_col=0)
            df.index = [str(x) for x in df.index]
            df.index = pd.to_datetime(df.index)
            last_date = df.index.tolist()[-1].strftime('%Y%m%d')
            if True:
                date_list_tmp = [date for date in all_days if last_date<date<=pre_date]
                df_tmp = eod_df_dict[factor]
                df_tmp.index = pd.to_datetime(df_tmp.index)
                remain_index = [d for d in df.index if d not in df_tmp.index]
                df = df.reindex(remain_index).append(df_tmp).sort_index()
                df.to_csv(file_name)
                print('数据{}更新完毕'.format(factor))
            else:
                print('数据{}已是最新'.format(factor))
        else:
            date_list_tmp = [date for date in all_days if init_start_date<=date<=pre_date]
            df_tmp = eod_df_dict[factor]
            df_tmp.index = pd.to_datetime(df_tmp.index)
            df_tmp.to_csv(file_name)
            print('数据{}更新完毕'.format(factor))
    return df_tmp



refresh_re_period_eod()
refresh_fin_byQ()
refresh_fin_eod_from_Q()

['20090930', '20091231', '20100331', '20100630', '20100930', '20101231', '20110331', '20110630', '20110930', '20111231', '20120331', '20120630', '20120930', '20121231', '20130331', '20130630', '20130930', '20131231', '20140331', '20140630', '20140930', '20141231', '20150331', '20150630', '20150930', '20151231', '20160331', '20160630', '20160930', '20161231', '20170331', '20170630', '20170930', '20171231', '20180331', '20180630', '20180930', '20181231', '20190331', '20190630', '20190930', '20191231', '20200331', '20200630', '20200930', '20201231', '20210331', '20210630', '20210930', '20211231', '20220331', '20220630', '20220930', '20221231', '20230331', '20230630']
get_re_period_dict......2010-01-04
get_re_period_dict......2011-01-14
get_re_period_dict......2012-01-31
get_re_period_dict......2013-02-04
get_re_period_dict......2014-02-25
get_re_period_dict......2015-03-05
get_re_period_dict......2016-03-11
get_re_period_dict......2017-03-21
get_re_period_dict......2018-03-28
get_re_perio

code,000001.SZ,000002.SZ,000003.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,...,A23263.SZ,A23264.SZ,A23265.SZ,A23266.SZ,A23267.SZ,A23268.SZ,A23269.SZ,A23270.SZ,A23271.SZ,A23272.SH
2010-12-01,20100930,20100930,20100630,20100930,20100930,20100930,20100930,20100930,20100930,20100930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-12-02,20100930,20100930,20100630,20100930,20100930,20100930,20100930,20100930,20100930,20100930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-12-03,20100930,20100930,20100630,20100930,20100930,20100930,20100930,20100930,20100930,20100930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-12-06,20100930,20100930,20100630,20100930,20100930,20100930,20100930,20100930,20100930,20100930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-12-07,20100930,20100930,20100630,20100930,20100930,20100930,20100930,20100930,20100930,20100930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-12-08,20100930,20100930,20100630,20100930,20100930,20100930,20100930,20100930,20100930,20100930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-12-09,20100930,20100930,20100630,20100930,20100930,20100930,20100930,20100930,20100930,20100930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-12-10,20100930,20100930,20100630,20100930,20100930,20100930,20100930,20100930,20100930,20100930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-12-13,20100930,20100930,20100630,20100930,20100930,20100930,20100930,20100930,20100930,20100930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-12-14,20100930,20100930,20100630,20100930,20100930,20100930,20100930,20100930,20100930,20100930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


['20000930', '20001231', '20010331', '20010630', '20010930', '20011231', '20020331', '20020630', '20020930', '20021231', '20030331', '20030630', '20030930', '20031231', '20040331', '20040630', '20040930', '20041231', '20050331', '20050630', '20050930', '20051231', '20060331', '20060630', '20060930', '20061231', '20070331', '20070630', '20070930', '20071231', '20080331', '20080630', '20080930', '20081231', '20090331', '20090630', '20090930', '20091231', '20100331', '20100630', '20100930', '20101231', '20110331', '20110630', '20110930', '20111231', '20120331', '20120630', '20120930', '20121231', '20130331', '20130630', '20130930', '20131231', '20140331', '20140630', '20140930', '20141231', '20150331', '20150630', '20150930', '20151231', '20160331', '20160630', '20160930', '20161231', '20170331', '20170630', '20170930', '20171231', '20180331', '20180630', '20180930', '20181231', '20190331', '20190630', '20190930', '20191231', '20200331', '20200630', '20200930', '20201231', '20210331', '20

code,000001.SZ,000002.SZ,000003.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,...,A23264.SZ,A23265.SZ,A23266.SZ,A23267.SZ,A23268.SZ,A23269.SZ,A23270.SZ,A23271.SZ,A23272.SH,T00018.SH
re_period,,,,,,,,,,,,,,,,,,,,,
2000-09-30,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2000-12-31,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2001-03-31,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2001-06-30,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2001-09-30,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2001-12-31,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2002-03-31,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2002-06-30,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2002-09-30,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


refresh_fin_eod_from_Q...:start_date:20101201,end_date:20230906


NameError: name 'get_single_factor_values' is not defined